<a href="https://colab.research.google.com/github/18wh1a1240/ML-Assignments-WISE/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Importing the Keras libraries and packages
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

In [19]:
!unzip "/content/drive/MyDrive/cats_and_dogs_filtered.zip" -d "/content"

Archive:  /content/drive/MyDrive/cats_and_dogs_filtered.zip
replace /content/cats_and_dogs_filtered/cats_and_dogs_filtered/train/cats/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
base_dir = os.path.join(os.getcwd(), '/content/cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

##Setting Model Parameters

In [ ]:
BATCH_SIZE = 100  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels

##Model Creation

In [22]:
# Initialising the CNN
classifier = Sequential()

In [23]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))

In [24]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(2, 2))

In [25]:
# Step 1 & 2 with more hidden layers
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(2,2))

classifier.add(Conv2D(128, (3,3), activation='relu'))
classifier.add(MaxPooling2D(2,2))

classifier.add(Conv2D(128, (3,3), activation='relu'))
classifier.add(MaxPooling2D(2,2))

In [26]:
# Step 3 - Dropout
classifier.add(Dropout(0.5))

In [27]:
#Adding a second convolutional layer classifier.add(Conv2D(32, (3, 3), activation = 'relu')) classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 4 - Flattening
classifier.add(Flatten())

In [28]:
# Step 5 - Full connection
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 2, activation = 'softmax'))

In [29]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [30]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

##Data Augumentation and Preparation

In [31]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [32]:
training_set = train_datagen.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=train_dir,
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='binary')

Found 2000 images belonging to 2 classes.


In [33]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [34]:
validation_set = validation_datagen.flow_from_directory(batch_size=BATCH_SIZE,
                                                        directory=validation_dir,
                                                        target_size=(IMG_SHAPE, IMG_SHAPE),
                                                        class_mode='binary')

Found 1000 images belonging to 2 classes.


##Train Model

In [35]:
epochs=100
history = classifier.fit_generator(
    training_set,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
20/20 [==============================] - 112s 6s/step - loss: 0.8384 - accuracy: 0.5193 - val_loss: 0.6910 - val_accuracy: 0.5900
Epoch 2/100
20/20 [==============================] - 110s 5s/step - loss: 0.6937 - accuracy: 0.5140 - val_loss: 0.6924 - val_accuracy: 0.5640
Epoch 3/100
20/20 [==============================] - 108s 5s/step - loss: 0.6921 - accuracy: 0.5108 - val_loss: 0.6900 - val_accuracy: 0.5270
Epoch 4/100
20/20 [==============================] - 108s 5s/step - loss: 0.6913 - accuracy: 0.5056 - val_loss: 0.6846 - val_accuracy: 0.5060
Epoch 5/100
20/20 [==============================] - 108s 5s/step - loss: 0.6863 - accuracy: 0.5338 - val_loss: 0.6497 - val_accuracy: 0.6140
Epoch 6/100
20/20 [==============================] - 109s 5s/step - loss: 0.6843 - accuracy: 0.5449 - val_loss: 0.6865 - val_accuracy: 0.5730
Epoch 7/100
20/20 [==============================] - 109s 5s/step - loss: 0.6923 - accuracy: 0.5537 - val_loss: 0.6819 - val_accuracy: 0.5440
Epoch 

##Training Visualization

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()